# Pair Programming Scenarios


#### Setup
Set the ~~MakerSuite~~ Gemini API key with the provided helper function.

In [1]:
from utils import get_api_key
# PaLM legacy
## import google.generativeai as palm
## palm.configure(api_key=get_api_key())

# Gemini API
import os
import google.generativeai as genai
from google.api_core import client_options as client_options_lib

genai.configure(
    api_key=get_api_key(),
    transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    )
)

#### Pick the model that generates text

```Python
# Legacy models shown in the video
models = [m for m in genai.list_models() if 'generateText' in m.supported_generation_methods]
model_bison = models[0]
# Model Bison set as legacy model in 2024
model_bison
```

In [2]:
# Set the model to connect to the Gemini API
model_flash = genai.GenerativeModel(model_name='gemini-2.0-flash')

### Legacy PaLM API
#### Helper function to call the PaLM API

```Python
from google.api_core import retry
@retry.Retry()
def generate_text(prompt, 
                  model=model_bison, 
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)
```

### Helper function to call the Gemini API

In [3]:
def generate_text(prompt,
                  model=model_flash,
                  temperature=0.0):
    return model_flash.generate_content(prompt,
                                  generation_config={'temperature':temperature})

### Scenario 1: Improve existing code
- An LLM can help you rewrite your code in the way that's recommended for that particular language.
- You can ask an LLM to rewrite your Python code in a way that is more 'Pythonic".

In [4]:
prompt_template = """
I don't think this code is the best way to do it in Python, can you help me?

{question}

**Please explain, in detail**, what you did to improve it.
"""

In [5]:
question = """
def func_x(array)
  for i in range(len(array)):
    print(array[i])
"""

In [6]:
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
# Gemini API
print(completion.text)

# PaLM legacy
## print(completion.result)

Okay, let's analyze the provided Python code and explore better, more Pythonic ways to achieve the same result.

**Original Code:**

```python
def func_x(array):
  for i in range(len(array)):
    print(array[i])
```

**Explanation of the Original Code:**

*   **`def func_x(array):`**: This defines a function named `func_x` that takes one argument, `array`.  We assume `array` is intended to be a list or other iterable.
*   **`for i in range(len(array)):`**: This is the core of the loop.
    *   `len(array)`:  Calculates the number of elements in the `array`.
    *   `range(len(array))`: Creates a sequence of numbers from 0 up to (but not including) the length of the array.  For example, if `array` has 5 elements, `range(len(array))` will produce the sequence `0, 1, 2, 3, 4`.
    *   `for i in ...`: The loop iterates through each number in the sequence, assigning it to the variable `i`.  `i` effectively becomes the *index* of the element we want to access in the array.
*   **`print(array

#### Ask for multiple ways of rewriting your code

In [7]:
prompt_template = """
I don't think this code is the best way to do it in Python, can you help me?

{question}

**Please explore multiple ways of solving the problem**, and explain each.
"""

In [8]:
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.text)

Okay, let's explore better ways to iterate through and print elements of an array (or more generally, an iterable) in Python.  The original code works, but it's not the most Pythonic or efficient in many cases.

**Original Code (and why it's not ideal):**

```python
def func_x(array):
  for i in range(len(array)):
    print(array[i])
```

*   **Not Pythonic:**  Python favors direct iteration over elements rather than using indices.  It's generally more readable and less prone to off-by-one errors.
*   **Potentially Less Efficient:**  `len(array)` is called in each iteration. While usually fast, it's unnecessary.  Accessing elements by index (`array[i]`) can be slightly slower than direct iteration, especially for some data structures (like linked lists, though Python lists are arrays).

**Better Alternatives:**

1.  **Direct Iteration (The Most Pythonic Way):**

    ```python
    def func_x(array):
      for element in array:
        print(element)
    ```

    *   **Explanation:** Thi

#### Paste markdown into a markdown cell

If the model outputs what looks like a table in markdown, you can copy-paste markdown into a markdown cell to make it easier to view:

For example:

| Method | Pros | Cons |
|---|---|---|
| List comprehension | Concise | Can be difficult to read for complex code |
| `enumerate()` | Easy to read | Requires an extra variable to store the index |
| `map()` | Flexible | Requires a custom function to format the output |


#### Ask the model to recommend one of the methods as most 'Pythonic'

In [9]:
prompt_template = """
I don't think this code is the best way to do it in Python, can you help me?

{question}

Please explore multiple ways of solving the problem, 
**and tell me which is the most Pythonic**
"""

In [10]:
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.text)

Okay, I can definitely help you explore better ways to iterate through and print elements of an array (or, more accurately in Python, a list) in Python.

**The Original Code (and its Issues)**

```python
def func_x(array):
  for i in range(len(array)):
    print(array[i])
```

While this code *works*, it's not considered the most Pythonic way to iterate through a list.  The main issue is that you're using an index (`i`) to access the elements.  Python's `for` loop is designed to directly iterate over the *elements* themselves, which is cleaner and more readable.

**More Pythonic Alternatives**

Here are several alternative ways to achieve the same result, along with explanations and why they are considered more Pythonic:

1. **Direct Iteration (Most Pythonic)**

   ```python
   def func_x(array):
       for element in array:
           print(element)
   ```

   * **Explanation:** This is the most common and preferred way to iterate through a list in Python.  The `for element in array:`

### Scenario 2: Simplify code
- Ask the LLM to perform a code review.
- Note that adding/removing newline characters may affect the LLM completion that gets output by the LLM.

In [11]:
# option 1
prompt_template = """
Can you please simplify this code for a linked list in Python?

{question}

Explain in detail what you did to modify it, and why.
"""

After you try option 1, you can modify it to look like option 2 (in this markdown cell) and see how it changes the completion.
```Python
# option 2
prompt_template = """
Can you please simplify this code for a linked list in Python? \n
You are an expert in Pythonic code.

{question}

Please comment each line in detail, \n
and explain in detail what you did to modify it, and why.
"""
```

In [25]:
question = """
class Node:
  def __init__(self, dataval=None):
    self.dataval = dataval
    self.nextval = None

class SLinkedList:
  def __init__(self):
    self.headval = None

# initialise the singly linked list - current node links to the next node
list1 = SLinkedList()

# add node1
list1.headval = Node("Mon")

# create nodes2/3
e2 = Node("Tue")
e3 = Node("Wed")

# link up the nodes
list1.headval.nextval = e2
e2.nextval = e3

"""

In [13]:
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.text)

```python
class Node:
  def __init__(self, data):
    self.data = data
    self.next = None

class LinkedList:
  def __init__(self):
    self.head = None

# Example Usage:
my_list = LinkedList()
my_list.head = Node("Mon")
my_list.head.next = Node("Tue")
my_list.head.next.next = Node("Wed")
```

Here's a breakdown of the changes and the reasoning behind them:

**1. Renaming Variables for Clarity and Consistency:**

*   `dataval` to `data`:  The term `data` is more standard and self-explanatory for representing the value stored within a node.  It's a common convention in data structures.
*   `nextval` to `next`:  Similarly, `next` is the universally accepted name for the pointer to the next node in a linked list.  Using `nextval` is less common and can be confusing.
*   `headval` to `head`:  Again, `head` is the standard name for the first node in a linked list.
*   `SLinkedList` to `LinkedList`:  The "S" in `SLinkedList` likely stood for "Singly" linked list.  While technically correct,

In [15]:
# try the model's ode
class Node:
  def __init__(self, data):
    self.data = data
    self.next = None

class LinkedList:
  def __init__(self):
    self.head = None

# Example Usage:
my_list = LinkedList()
my_list.head = Node("Mon")
my_list.head.next = Node("Tue")
my_list.head.next.next = Node("Wed")
my_list

In [18]:
my_list.head.data

'Mon'

In [19]:
my_list.head.next.data

'Tue'

In [20]:
my_list.head.next.next.data

'Wed'

#### Option 2

In [26]:
# option 2
prompt_template2 = """
Can you please simplify this code for a linked list in Python? \n
**You are an expert in Pythonic code.**

{question}

Please comment each line in detail, \n
and explain in detail what you did to modify it, and why.
"""
completion = generate_text(
    prompt = prompt_template2.format(question=question)
)
print(completion.text)

```python
class Node:
    """Represents a node in the linked list."""
    def __init__(self, data=None):
        """Initializes a new node with optional data."""
        self.data = data  # Data stored in the node
        self.next = None  # Pointer to the next node in the list (initially None)


class LinkedList:
    """Represents a singly linked list."""
    def __init__(self):
        """Initializes an empty linked list."""
        self.head = None  # The head of the list (initially None)


# Initialize the linked list
my_list = LinkedList()

# Add the first node
my_list.head = Node("Mon")

# Add the second node
my_list.head.next = Node("Tue")

# Add the third node
my_list.head.next.next = Node("Wed")
```

**Explanation of Changes and Simplifications:**

1.  **Class Naming:**
    *   `SLinkedList` was changed to `LinkedList`.  The "S" for "Singly" is redundant in the class name, as the implementation already implies it's a singly linked list.  Using `LinkedList` is more concise and 

### Scenario 3: Write test cases

- It may help to specify that you want the LLM to output "in code" to encourage it to write unit tests instead of just returning test cases in English.

In [21]:
prompt_template = """
Can you please create test cases in code for this Python code?

{question}

Explain in detail what these test cases are designed to achieve.
"""

In [22]:
# Note that the code I'm using here was output in the previous
# section. Your output code may be different.
question = """
class Node:
  def __init__(self, dataval=None):
    self.dataval = dataval
    self.nextval = None

class SLinkedList:
  def __init__(self):
    self.head = None

def create_linked_list(data):
  head = Node(data[0])
  for i in range(1, len(data)):
    node = Node(data[i])
    node.nextval = head
    head = node
  return head

list1 = create_linked_list(["Mon", "Tue", "Wed"])
"""

In [23]:
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.text)

```python
import unittest

class Node:
  def __init__(self, dataval=None):
    self.dataval = dataval
    self.nextval = None

class SLinkedList:
  def __init__(self):
    self.head = None

def create_linked_list(data):
  head = Node(data[0])
  for i in range(1, len(data)):
    node = Node(data[i])
    node.nextval = head
    head = node
  return head


class TestCreateLinkedList(unittest.TestCase):

    def test_empty_list(self):
        """Test creating a linked list from an empty list."""
        head = create_linked_list([])
        self.assertIsNone(head)

    def test_single_element_list(self):
        """Test creating a linked list from a list with a single element."""
        head = create_linked_list(["Mon"])
        self.assertEqual(head.dataval, "Mon")
        self.assertIsNone(head.nextval)

    def test_multiple_elements_list(self):
        """Test creating a linked list from a list with multiple elements."""
        head = create_linked_list(["Mon", "Tue", "Wed"])
       

In [27]:
import unittest

class Node:
  def __init__(self, dataval=None):
    self.dataval = dataval
    self.nextval = None

class SLinkedList:
  def __init__(self):
    self.head = None

def create_linked_list(data):
  head = Node(data[0])
  for i in range(1, len(data)):
    node = Node(data[i])
    node.nextval = head
    head = node
  return head


class TestCreateLinkedList(unittest.TestCase):

    def test_empty_list(self):
        """Test creating a linked list from an empty list."""
        head = create_linked_list([])
        self.assertIsNone(head)

    def test_single_element_list(self):
        """Test creating a linked list from a list with a single element."""
        head = create_linked_list(["Mon"])
        self.assertEqual(head.dataval, "Mon")
        self.assertIsNone(head.nextval)

    def test_multiple_elements_list(self):
        """Test creating a linked list from a list with multiple elements."""
        head = create_linked_list(["Mon", "Tue", "Wed"])
        self.assertEqual(head.dataval, "Wed")
        self.assertEqual(head.nextval.dataval, "Tue")
        self.assertEqual(head.nextval.nextval.dataval, "Mon")
        self.assertIsNone(head.nextval.nextval.nextval)

    def test_different_data_types(self):
        """Test creating a linked list with different data types."""
        head = create_linked_list([1, "hello", 3.14])
        self.assertEqual(head.dataval, 3.14)
        self.assertEqual(head.nextval.dataval, "hello")
        self.assertEqual(head.nextval.nextval.dataval, 1)
        self.assertIsNone(head.nextval.nextval.nextval)

    def test_duplicate_elements(self):
        """Test creating a linked list with duplicate elements."""
        head = create_linked_list(["A", "B", "A", "C"])
        self.assertEqual(head.dataval, "C")
        self.assertEqual(head.nextval.dataval, "A")
        self.assertEqual(head.nextval.nextval.dataval, "B")
        self.assertEqual(head.nextval.nextval.nextval.dataval, "A")
        self.assertIsNone(head.nextval.nextval.nextval.nextval)

    def test_long_list(self):
        """Test creating a linked list from a long list to check for performance and stack overflow issues."""
        data = list(range(1000))
        head = create_linked_list(data)
        # Only check the first and last few elements to avoid excessive assertions
        self.assertEqual(head.dataval, 999)
        self.assertEqual(head.nextval.dataval, 998)
        # Traverse to the end (or near the end) to check the structure
        current = head
        for _ in range(997):
            current = current.nextval
        self.assertEqual(current.dataval, 2)
        self.assertEqual(current.nextval.dataval, 1)
        self.assertEqual(current.nextval.nextval.dataval, 0)
        self.assertIsNone(current.nextval.nextval.nextval)


if __name__ == '__main__':
    unittest.main()

E
ERROR: /home/jovyan/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/home/jovyan/'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/usr/local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Scenario 4: Make code more efficient
- Improve runtime by potentially avoiding inefficient methods (such as ones that use recursion when not needed).

In [28]:
prompt_template = """
Can you please make this code more efficient?

{question}


# 'decorator'
Explain in detail what you changed and why.
"""

In [29]:
question = """
# Returns index of x in arr if present, else -1
def binary_search(arr, low, high, x):
    # Check base case
    if high >= low:
        mid = (high + low) // 2
        if arr[mid] == x:
            return mid
        elif arr[mid] > x:
            return binary_search(arr, low, mid - 1, x)
        else:
            return binary_search(arr, mid + 1, high, x)
    else:
        return -1

# Test array
arr = [ 2, 3, 4, 10, 40 ]
x = 10

# Function call
result = binary_search(arr, 0, len(arr)-1, x)

if result != -1:
    print("Element is present at index", str(result))
else:
    print("Element is not present in array")

"""

In [30]:
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.text)

```python
# Returns index of x in arr if present, else -1
def binary_search(arr, low, high, x):
    """
    Performs a binary search on a sorted array.

    Args:
        arr: The sorted array to search.
        low: The starting index of the search range.
        high: The ending index of the search range.
        x: The element to search for.

    Returns:
        The index of x in arr if present, else -1.
    """
    while low <= high:
        mid = low + (high - low) // 2  # Prevent potential overflow
        if arr[mid] == x:
            return mid
        elif arr[mid] < x:
            low = mid + 1
        else:
            high = mid - 1
    return -1

# Test array
arr = [ 2, 3, 4, 10, 40 ]
x = 10

# Function call
result = binary_search(arr, 0, len(arr)-1, x)

if result != -1:
    print("Element is present at index", str(result))
else:
    print("Element is not present in array")
```

Here's a breakdown of the changes and the reasoning behind them:

**1. Iterative Approach Inst

#### Try out the LLM-generated code
- If it uses `bisect`, you may first need to `import bisect`
- Remember to check what the generated code is actually doing.  For instance, the code may work because it is calling a predefined function (such as `bisect`), even though the rest of the code is technically broken.

In [ ]:
# Paste the LLM-generated code to inspect and debug it


In [33]:
# Returns index of x in arr if present, else -1
def binary_search(arr, low, high, x):
    """
    Performs a binary search on a sorted array.

    Args:
        arr: The sorted array to search.
        low: The starting index of the search range.
        high: The ending index of the search range.
        x: The element to search for.

    Returns:
        The index of x in arr if present, else -1.
    """
    while low <= high:
        mid = low + (high - low) // 2  # Prevent potential overflow
        if arr[mid] == x:
            return mid
        elif arr[mid] < x:
            low = mid + 1
        else:
            high = mid - 1
    return -1

# Test array
arr = [ 2, 3, 4, 10, 40 ]
x = 10

# Function call
result = binary_search(arr, 0, len(arr)-1, x)

if result != -1:
    print("Element is present at index", str(result))
else:
    print("Element is not present in array")

Element is present at index 3


In [34]:
# Test array
arr = [ 2, 3, 4, 10, 40 ]
x = 14

# Function call
result = binary_search(arr, 0, len(arr)-1, x)

if result != -1:
    print("Element is present at index", str(result))
else:
    print("Element is not present in array")

Element is not present in array


### Scenario 5: Debug your code

In [35]:
prompt_template = """
Can you please help me to debug this code?

{question}

Explain in detail what you found and why it was a bug.
"""

In [36]:
# I deliberately introduced a bug into this code! 
# Let's see if the LLM can find it.
# Note -- the model can't see this comment -- but the bug is in the
# 'print' function. 
# There's a circumstance where nodes can be null, and trying
# to print them would give a null error.
question = """
class Node:
   def __init__(self, data):
      self.data = data
      self.next = None
      self.prev = None

class doubly_linked_list:
   def __init__(self):
      self.head = None

# Adding data elements
   def push(self, NewVal):
      NewNode = Node(NewVal)
      NewNode.next = self.head
      if self.head is not None:
         self.head.prev = NewNode
      self.head = NewNode

# Print the Doubly Linked list in order
   def listprint(self, node):
       print(node.data),
       last = node
       node = node.next

dllist = doubly_linked_list()
dllist.push(12)
dllist.push(8)
dllist.push(62)
dllist.listprint(dllist.head)

"""

Notice in this case that we are using the default temperature of `0.7` to generate the example that you're seeing in the lecture video.  
- Since a temperature > 0 encourages more randomness in the LLM output, you may want to run this code a couple times to see what it outputs.

In [37]:
completion = generate_text(
    prompt = prompt_template.format(question=question),
    temperature = 0.7
)
print(completion.text)

Okay, let's analyze the provided code and identify the bug.

**Code:**

```python
class Node:
   def __init__(self, data):
      self.data = data
      self.next = None
      self.prev = None

class doubly_linked_list:
   def __init__(self):
      self.head = None

# Adding data elements
   def push(self, NewVal):
      NewNode = Node(NewVal)
      NewNode.next = self.head
      if self.head is not None:
         self.head.prev = NewNode
      self.head = NewNode

# Print the Doubly Linked list in order
   def listprint(self, node):
       print(node.data), # Python 2 style print statement
       last = node
       node = node.next

dllist = doubly_linked_list()
dllist.push(12)
dllist.push(8)
dllist.push(62)
dllist.listprint(dllist.head)
```

**The Bug:**

The `listprint` method only prints the first node in the list. It does not iterate through the rest of the list.

**Explanation:**

The `listprint` function is designed to print the doubly linked list. However, within the `listprint`

In [38]:
completion = generate_text(
    prompt = prompt_template.format(question=question),
    temperature = 0.7
)
print(completion.text)

Okay, let's debug this Python code for a doubly linked list.

**The Problem**

The primary issue is in the `listprint` method.  It only prints the first element of the list and then stops. It also has an error in the print statement.

**Detailed Explanation**

1.  **Incorrect Loop Termination:** The `listprint` function has a single `print` statement and then sets `last = node` and `node = node.next`.  It doesn't have a loop to iterate through the entire linked list.  Therefore, it only processes the first node (the head).

2.  **Missing Loop Condition:** To print all elements, you need a loop that continues as long as there are more nodes in the list. The loop should stop when `node` becomes `None`.

3. **Incorrect Print Statement:** The print statement `print(node.data),` is using Python 2 syntax. In Python 3, this will result in a `SyntaxError: invalid syntax`.  We need to use `print(node.data, end=" ")` to achieve the same effect of printing on the same line.

**Corrected Code**

`

In [39]:
completion = generate_text(
    prompt = prompt_template.format(question=question),
    temperature = 0
)
print(completion.text)

Okay, let's analyze the code and identify the bug.

**Code:**

```python
class Node:
   def __init__(self, data):
      self.data = data
      self.next = None
      self.prev = None

class doubly_linked_list:
   def __init__(self):
      self.head = None

# Adding data elements
   def push(self, NewVal):
      NewNode = Node(NewVal)
      NewNode.next = self.head
      if self.head is not None:
         self.head.prev = NewNode
      self.head = NewNode

# Print the Doubly Linked list in order
   def listprint(self, node):
       print(node.data),  # Python 2 style print, will cause issues in Python 3
       last = node
       node = node.next

dllist = doubly_linked_list()
dllist.push(12)
dllist.push(8)
dllist.push(62)
dllist.listprint(dllist.head)
```

**The Bug and Explanation:**

The primary bug lies within the `listprint` method.  It only prints the first node of the linked list and then stops.  It doesn't iterate through the entire list.  Also, the print statement is using Pytho

#### Reminder to check the code
You can use an  LLM to give you insights and check for blind spots, but remember to check that the generated code is doing what you want it to do.